# 4. lekce: Cvičení
## Přijímací zkouška

Uvažujme nyní vyhodnocení přijímací zkoušky na střední školu technického zaměření, které jsou uloženy v souboru [prijimaci_zkousky.csv](data/prijimaci_zkousky.csv). Uchazeči a uchazečky musejí absolvovat tři písemné testy: test z anglického jazyka, matematiky a českého jazyka. Pokud dostane z některého z testů méně než 60 bodů, dostane hodnocení 0. Pokud má ze všech testů více než 60 bodů, je jeho skóre rovno součtu bodů ze všech tří testů. Dále uvažujeme, že za aktivity uchazeče přičítáme bonusové body:

- za 1. až 10. místo v některé z olympiád 10 bonusových bodů,
- za účast na letní škole dostane 5 bonusových bodů.

Vytvoř funkci `get_points()`, která přidělí každému z uchazečů počet bodů na základě daných kritérií. Dále vytvoř novou tabulku, která obsahuje pouze uchazeče a uchazečky, kteří mají nenulové hodnocení. Dále využij metodu `rank()` k výpočtu pořadí uchazeče/uchazečky v rámci daného oboru, a data seřaď podle oboru a počtu bodů. Nakonec vytvoř tabulku s přijatými uchazeči, kde budou uchazeči hodnocení od 1. do 30. místa pro daný obor.

Níže jsou data. Informace o olympiádách jsou ve sloupci `souteze`. Jsou tam uvedeny pouze údaje, které se započítávají do zkoušky, tj. získání 1. až 10. místa.

In [53]:
import pandas

data = pandas.read_csv("data/prijimaci_zkousky.csv")
data.head()

,prijmeni,krestni_jmeno,obor,body_aj,body_mat,body_cj,letni_skola,souteze
0,Květoslava,Růžičková,Elektrotechnika,62,48,51,ne,NaN
1,Jan,Moravec,Technické lyceum,81,74,66,ne,7. místo (matematická olympiáda)
2,Denis,Fiala,Informatika,63,69,65,ne,NaN
3,Luboš,Šimek,Informatika,78,79,65,ne,NaN
4,Vilém,Novotný,Informatika,84,68,78,ano,NaN


- Vytvoř funkci `get_points()`, která přidělí každému z uchazečů počet bodů na základě daných kritérií.
        
  - Pokud dostane z některého z testů méně než 60 bodů, dostane hodnocení 0. Pokud má ze všech testů více než 60 bodů, je jeho skóre rovno součtu bodů ze všech tří testů.
  - za 1. až 10. místo v některé z olympiád 10 bonusových bodů,
  - za účast na letní škole dostane 5 bonusových bodů.

In [54]:
def get_points(row):
    if row[["body_aj", "body_mat", "body_cj"]].min() < 60:
        return 0
    skore = row[["body_aj", "body_mat", "body_cj"]].sum()
    if not pandas.isnull(row["souteze"]):
        skore += 10
        # ...stejné jako skore = skore + 10, ale preferovanější zápis je +=
    if row["letni_skola"] == "ano":
        skore += 5
    return skore
    
data["body"] = data.apply(get_points, axis=1)
# ... Funkci get_points předáváme bez kulatých závorek, tzn. nevoláme ji, ale jen na ni odkazujeme
# ... axis=1: aplikovat po řádcích (po sloupcích by bylo axis=0)

data.head()

,prijmeni,krestni_jmeno,obor,body_aj,body_mat,body_cj,letni_skola,souteze,body
0,Květoslava,Růžičková,Elektrotechnika,62,48,51,ne,NaN,0
1,Jan,Moravec,Technické lyceum,81,74,66,ne,7. místo (matematická olympiáda),231
2,Denis,Fiala,Informatika,63,69,65,ne,NaN,197
3,Luboš,Šimek,Informatika,78,79,65,ne,NaN,222
4,Vilém,Novotný,Informatika,84,68,78,ano,NaN,235


- Dále vytvoř novou tabulku, která obsahuje pouze uchazeče a uchazečky, kteří mají nenulové hodnocení.

In [55]:
data = data[data["body"] > 0]
data = data.reset_index()

data.head()

,index,prijmeni,krestni_jmeno,obor,body_aj,body_mat,body_cj,letni_skola,souteze,body
0,1,Jan,Moravec,Technické lyceum,81,74,66,ne,7. místo (matematická olympiáda),231
1,2,Denis,Fiala,Informatika,63,69,65,ne,NaN,197
2,3,Luboš,Šimek,Informatika,78,79,65,ne,NaN,222
3,4,Vilém,Novotný,Informatika,84,68,78,ano,NaN,235
4,5,Drahomíra,Štěpánková,Elektrotechnika,69,74,60,ne,NaN,203


- Dále využij metodu `rank()` k výpočtu pořadí uchazeče/uchazečky v rámci daného oboru, a data seřaď podle oboru a počtu bodů.

In [56]:
# Vytvoříme pořadí pro každý obor:
# ...method="min": v případě shody bodů u více řádků přiřadí všem shodně nejmenší hodnotu pořadí
# ...ascending=False: sestupně, tzn. nejvíc bodů = 1. místo
data["poradi"] = data.groupby("obor")["body"].rank(method="min", ascending=False)
data = data.sort_values(["obor", "poradi"])

data.tail()

,index,prijmeni,krestni_jmeno,obor,body_aj,body_mat,body_cj,letni_skola,souteze,body,poradi
54,87,Veronika,Poláková,Technické lyceum,68,68,61,ano,NaN,202,27.0
25,38,Monika,Kratochvílová,Technické lyceum,60,76,65,ne,NaN,201,28.0
11,15,Erik,Dvořák,Technické lyceum,68,71,61,ne,NaN,200,29.0
119,195,Naděžda,Čermáková,Technické lyceum,71,61,67,ne,NaN,199,30.0
21,31,Jan,Kadlec,Technické lyceum,61,67,62,ne,NaN,190,31.0


- Nakonec vytvoř tabulku s přijatými uchazeči, kde budou uchazeči hodnocení od 1. do 30. místa pro daný obor.

In [57]:
def prijat(row):
    if row["poradi"] <= 30:
        return "Ano"
    else:
        return "Ne"
# ...Zde je to pro názornost "Ano"/"Ne", ale jinak by se pro případné další zpracování spíš hodilo použít hodnoty True/False

data["prijat"] = data.apply(prijat, axis=1)

# Alternativa z předchozích lekcí:
data["prijat_cut"] = pandas.cut(data["poradi"], bins=[0,30,float("inf")], labels=["Ano", "Ne"])

data.tail()

,index,prijmeni,krestni_jmeno,obor,body_aj,body_mat,body_cj,letni_skola,souteze,body,poradi,prijat,prijat_cut
54,87,Veronika,Poláková,Technické lyceum,68,68,61,ano,NaN,202,27.0,Ano,Ano
25,38,Monika,Kratochvílová,Technické lyceum,60,76,65,ne,NaN,201,28.0,Ano,Ano
11,15,Erik,Dvořák,Technické lyceum,68,71,61,ne,NaN,200,29.0,Ano,Ano
119,195,Naděžda,Čermáková,Technické lyceum,71,61,67,ne,NaN,199,30.0,Ano,Ano
21,31,Jan,Kadlec,Technické lyceum,61,67,62,ne,NaN,190,31.0,Ne,Ne


## Bonusy
- Zájemcům o studium na oborech Elektrotechnika a Informatika přiděluj bonusové body pouze za fyzikální nebo matematickou olympiádu, zájemcům o lyceum i za ostatní olympiády.
- Za 1. až 3. místo v olympiádě přiděluj 15 bodů, za nižší místa 5.

In [58]:
import re

def get_points(row):

    if row[["body_aj", "body_mat", "body_cj"]].min() < 60:
        return 0
    skore = row[["body_aj", "body_mat", "body_cj"]].sum()

    if not pandas.isnull(row["souteze"]):
        # Body za olympiádu
        # Extrahujeme část řetězce před tečkou, tedy pořadí
        olympiada_umisteni = row["souteze"].split(".")[0]
        if olympiada_umisteni in ["1", "2", "3"]:
            body_olympiada = 15
        else:
            body_olympiada = 5

        # Body za kombinaci oboru a olympiády
        obor_je_lyceum = row["obor"] == "Technické lyceum"
        obor_je_technicky = row["obor"] in ["Elektrotechnika", "Informatika"]
        # Extrahujeme název olympiády mezi závorkami pomocí split(), případně regulárního výrazu
        olympiada_nazev = row["souteze"].split("(")[1][:-1]
        # olympiada_nazev = re.findall(r'\(.*\)', row["souteze"])[0][1:-1]
        # print(olympiada_nazev)
        olympiada_je_matfyz = olympiada_nazev in ["matematická olympiáda", "fyzikální olympiáda"]
        olympiada_je_matfyz
        if obor_je_lyceum or (obor_je_technicky and olympiada_je_matfyz):
            skore = skore + body_olympiada
    
    if row["letni_skola"] == "ano":
        skore = skore + 5
    return skore


data["body2"] = data.apply(get_points, axis=1)
data.head()
# data[~pandas.isnull(data["souteze"])]

,index,prijmeni,krestni_jmeno,obor,body_aj,body_mat,body_cj,letni_skola,souteze,body,poradi,prijat,prijat_cut,body2
10,12,Luděk,Kratochvíl,Elektrotechnika,92,89,93,ne,NaN,274,1.0,Ano,Ano,274
89,143,Daniela,Poláková,Elektrotechnika,102,78,94,ne,NaN,274,1.0,Ano,Ano,274
8,10,Vilém,Dvořák,Elektrotechnika,98,92,80,ne,NaN,270,3.0,Ano,Ano,270
98,160,Barbora,Vacková,Elektrotechnika,90,83,87,ne,2. místo (přírodopisná olympiáda),270,3.0,Ano,Ano,260
45,72,Lubomír,Holub,Elektrotechnika,96,90,82,ne,NaN,268,5.0,Ano,Ano,268
